<a href="https://colab.research.google.com/github/advait-t/Artificial-Intelligence/blob/master/jio_automating_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        # print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        #     "There are " + str(mis_val_table_ren_columns.shape[0]) +
        #       " columns that have missing values.")
        return mis_val_table_ren_columns

In [4]:
def drop_na(data):   
    missing_values = pd.DataFrame(missing_values_table(data))
    for i in range(len(missing_values['% of Total Values'])):
        if missing_values['% of Total Values'][i] <= 20:
            cols = list(missing_values.index.values)
            data.dropna(subset = cols, inplace=True)
    drop_thresh = data.shape[0]*.9
    data = data.dropna(thresh=drop_thresh, how='all', axis='columns').copy()
    return data

Making a generalised apprach for unique values

In [5]:
def unique_value_count(data):
    unique_counts = pd.DataFrame.from_records([(col, data[col].nunique()) for col in data.columns],columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
    return unique_counts

In [6]:
def convert_dtype(data,unique=0):
    unique_counts = pd.DataFrame.from_records([(col, data[col].nunique()) for col in data.columns],columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])
    if unique == 0:
        unique = 40
    for i in data.columns:
        if data[i].nunique() <= unique:
            data[i] = data[i].astype('object')
        else: #data.dtypes[i] != object:
            data[i] = data[i].astype('float64')
    return data

In [7]:
def columns_select(data,exclude):
    data = data.select_dtypes(exclude=[exclude])
    return data

In [8]:
def read_data(path):
    if path.lower().endswith(('.csv')):
        data = pd.read_csv(path)
    else:
        data = pd.read_parquet(path, engine='pyarrow')
    return data

In [9]:
def col_lis(data):
    column_names = data.columns
    return column_names

In [10]:
def display_bar_categorical(data):
    data = columns_select(data,'float64')
    data = columns_select(data,'int64')
    column_names = data.columns
    try:
        for i in range(len(column_names)):
            #dfg = data1.groupby(column_names[i]).count()

            fig = px.histogram(data, x = column_names[i-1], color=column_names[i-1]) # add percentage values for each
            #fig = px.bar(dfg, x=column_names[i-1], y=column_names[i-1])
            fig.show()
    except:
        pass
    return data

In [11]:
def check_special_char(data):
    special_char = ['?', '!','-','*']
    data = data.replace(special_char, np.nan)
    return data

In [12]:
def label_encoding(data):
    categorical_feature_mask = data.dtypes==object
    categorical_cols = data.columns[categorical_feature_mask].tolist()
    le = LabelEncoder()
    data[categorical_cols] = data[categorical_cols].apply(lambda col: le.fit_transform(col))
    return data, categorical_cols

In [13]:
def drop_columns(data,column_names):
    data = data.drop(column_names, axis = 1)
    return data

In [14]:
def create_bins(data,old_column_names,new_column_names,drop_old_column,bins, labels):
    bins_items = list(bins.values())[0]
    labels_items = list(labels.values())[0]

    for i in range(len(old_column_names)):
        bins_items = list(bins.values())[i]
        labels_items = list(labels.values())[i]
        data[new_column_names[i]] = pd.cut(data[old_column_names[i]], bins = bins_items, labels = labels_items)
    if drop_old_column == 'yes':
        data=data.drop(old_column_names, axis = 1)
    else:
        pass
    return data

In [15]:
#standard scaler for continuous variables
def scaler_int(data):
    continuous_feature_mask = data.dtypes== int
    #continuous_feature_mask1 = data.dtypes== float
    continuous_cols = data.columns[continuous_feature_mask].tolist()
    #continuous_cols1 = data.columns[continuous_feature_mask1].tolist()
    sc = StandardScaler()
    data[continuous_cols] = data[continuous_cols].apply(lambda col: sc.fit_transform(col))
    #data[continuous_cols1] = data[continuous_cols1].apply(lambda col1: sc.fit_transform(col1))
    return data, continuous_cols

In [16]:
def scaler_float(data):
    continuous_feature_mask = data.dtypes== int
    continuous_cols = data.columns[continuous_feature_mask].tolist()
    data1 = data.drop(continuous_cols, axis = 1)
    sc = StandardScaler()
    data1 = sc.fit_transform(data1)
    data1 = pd.DataFrame(data1)
    data[continuous_cols].update(data1)
    data = pd.DataFrame(data)
    return data, continuous_cols

In [17]:
data = read_data('/content/drive/MyDrive/Jio/tmp/digitaltwinoct19102020withappdata')

In [18]:
column_names = ['context_time_5min','entity_user_imsi', 'entity_cell_id']
data = drop_columns(data,column_names)

In [19]:
data.shape

(1551791, 16)

In [20]:
data = check_special_char(data)

In [21]:
missing_values_table(data)

,Missing Values,% of Total Values


In [22]:
data = drop_na(data)

In [23]:
missing_values_table(data)

,Missing Values,% of Total Values


In [24]:
old_column_names = ['customer_download_throughput']
new_column_names = ['customer_download_throughput_binned']
drop_old_column = 'yes'
bins = {"1":[0,500,int(max(data['customer_download_throughput']))]}
labels = {"1":[1,0]}
data = create_bins(data,old_column_names,new_column_names,drop_old_column,bins, labels)

In [25]:
data.head()

,entity_user_device,application_group,application_name,quantity_lsr_cqi,quantity_lsr_rsrp,quantity_lsr_rsrq,quantity_lsr_pusch_sinr,quantity_lsr_uplink_interference,quantity_lsr_ta,quantity_cell_probes_hsi_data_download_bytes,quantity_cell_probes_hsi_data_download_duration_millisecs,quantity_probes_hsi_data_download_bytes,quantity_probes_hsi_data_download_duration_millisecs,Hour,cell_download_throughput,customer_download_throughput_binned
0,MDI2,WEB,izatcloud,7.000,-85.000,-17.333,20.150,-116.885,115.333,5476583.0,14442.0,5476583.0,14442.0,0,3034.0,0
1,CPH1823,Social,Whatsapp,11.000,-98.000,-11.000,15.300,-116.070,312.000,8170401.0,24199.0,8170401.0,24199.0,0,2701.0,0
2,1606,WEB,HTTPS,7.000,-106.000,-17.500,0.700,-113.550,234.000,67453891.0,253889.0,3601553.0,80525.0,0,2125.0,1
3,SM-T295,STREAMING,NetFlixVideo,8.600,-99.000,-12.700,14.100,-116.660,217.000,3616392.0,25739.0,3616392.0,25739.0,0,1124.0,0
4,M1803E6I,STREAMING,Youtube,10.600,-108.067,-10.433,12.500,-118.847,358.933,1511896.0,2387.0,1511896.0,2387.0,0,5067.0,0


In [26]:
unique_value_count(data)

,Column_Name,Num_Unique
15,customer_download_throughput_binned,2
13,Hour,24
1,application_group,36
2,application_name,469
0,entity_user_device,2360
3,quantity_lsr_cqi,8683
5,quantity_lsr_rsrq,8870
7,quantity_lsr_uplink_interference,14834
14,cell_download_throughput,17324
6,quantity_lsr_pusch_sinr,22559


In [27]:
data = convert_dtype(data,2360)

In [28]:
# data_category = display_bar_categorical(data)

In [29]:
data, categorical_cols = label_encoding(data)

In [30]:
data.head()

,entity_user_device,application_group,application_name,quantity_lsr_cqi,quantity_lsr_rsrp,quantity_lsr_rsrq,quantity_lsr_pusch_sinr,quantity_lsr_uplink_interference,quantity_lsr_ta,quantity_cell_probes_hsi_data_download_bytes,quantity_cell_probes_hsi_data_download_duration_millisecs,quantity_probes_hsi_data_download_bytes,quantity_probes_hsi_data_download_duration_millisecs,Hour,cell_download_throughput,customer_download_throughput_binned
0,1288,34,400,7.0,-85.000,-17.333,20.15,-116.885,115.333,5476583.0,14442.0,5476583.0,14442.0,0,3034.0,0
1,493,30,328,11.0,-98.000,-11.000,15.30,-116.070,312.000,8170401.0,24199.0,8170401.0,24199.0,0,2701.0,0
2,9,34,141,7.0,-106.000,-17.500,0.70,-113.550,234.000,67453891.0,253889.0,3601553.0,80525.0,0,2125.0,1
3,1913,28,206,8.6,-99.000,-12.700,14.10,-116.660,217.000,3616392.0,25739.0,3616392.0,25739.0,0,1124.0,0
4,1150,28,346,10.6,-108.067,-10.433,12.50,-118.847,358.933,1511896.0,2387.0,1511896.0,2387.0,0,5067.0,0


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1551791 entries, 0 to 1551790
Data columns (total 16 columns):
 #   Column                                                     Non-Null Count    Dtype  
---  ------                                                     --------------    -----  
 0   entity_user_device                                         1551791 non-null  int64  
 1   application_group                                          1551791 non-null  int64  
 2   application_name                                           1551791 non-null  int64  
 3   quantity_lsr_cqi                                           1551791 non-null  float64
 4   quantity_lsr_rsrp                                          1551791 non-null  float64
 5   quantity_lsr_rsrq                                          1551791 non-null  float64
 6   quantity_lsr_pusch_sinr                                    1551791 non-null  float64
 7   quantity_lsr_uplink_interference                           1551791 non-n

In [32]:
data = scaler_float(data)

In [33]:
data = pd.DataFrame(data)

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [34]:
data.head()

,0
0,entity_user_device ... customer_dow...
1,"[entity_user_device, application_group, applic..."


In [35]:
columns = data.columns
for i in range(len(columns)):
    fig = px.histogram(data, x=data[columns[i]], histnorm='probability density')
    fig.show()

TypeError: ignored

Similarly for Time Sense Data, and numerical only.<br> Time Series Plots

Univariate- With Labels<br>